In [215]:
FILE = "example1.txt"

In [216]:
from collections import namedtuple
from typing import Literal, get_args, cast
from dataclasses import dataclass, field


Symbol = Literal["|", "-", "L", "J", "7", "F", ".", "S", "X"]

class Point(namedtuple("Point", ["y", "x"])):
    def __add__(self, other):
        return Point(self.y + other.y, self.x + other.x)
    
    def __sub__(self, other):
        return Point(self.y - other.y, self.x - other.x)
        


ADJACENT_COORDINATES = [
    Point(-1,  0),
    Point( 1,  0),
    Point( 0, -1),
    Point( 0,  1)
]

CONNECTS_TO: dict[Symbol, tuple[Point, Point] | tuple] = {
    "|": (Point(-1,  0), Point(1,  0)),
    "-": (Point( 0, -1), Point(0,  1)),
    "L": (Point(-1,  0), Point(0,  1)),
    "J": (Point(-1,  0), Point(0, -1)),
    "7": (Point( 1,  0), Point(0, -1)),
    "F": (Point( 1,  0), Point(0,  1)),
    ".": ()
}

CONNECTED_TO_SYMBOL = {
    tuple(sorted(connected)): symbol
    for symbol, connected in CONNECTS_TO.items()
}

STARTING_CHAR = "S"

@dataclass(eq=True, frozen=True)
class MapTile:
    symbol: Symbol = field(hash=True)
    point: Point = field(hash=True)
    is_loop: bool = field(default=False, hash=False)
    is_added: bool = field(default=False, hash=False)

    def set_loop(self, is_loop: bool):
        return MapTile(self.symbol, self.point, is_loop)
    
    def __eq__(self, __value: object) -> bool:
        if isinstance(__value, MapTile):
            return self.symbol == __value.symbol and self.point == __value.point
        return False
    
    def __str__(self) -> str:
        return str(self.symbol)

    def __repr__(self) -> str:
        return f"MapTile({self.symbol} @ (y={self.point.y}, x={self.point.x}))"


class Map(list[list[MapTile]]):
    def __init__(self, iterable):
        self.min_y = 0
        self.max_y = len(iterable) - 1
        self.min_x = 0
        self.max_x = len(iterable[0]) - 1
        super().__init__(iterable)
    
    def all_tiles(self):
        for row in self:
            for tile in row:
                yield tile

    def __getitem__(self, index):
        if isinstance(index, Point):
            y, x = index
            return self[y][x]
        return super().__getitem__(index)
    
    def __setitem__(self, index, value):
        print(index, value)
        if isinstance(index, Point):
            y, x = index
            row = self[y]
            row[x] = value
            return
        super().__setitem__(index, value)
    
    def point_valid(self, point: Point) -> bool:
        return (self.min_y <= point.y <= self.max_y) and (self.min_x <= point.x <= self.max_x)
    
    def get_adjacent(self, tile: MapTile) -> list[MapTile]:
        point = tile.point
        adjacent = [point + adj for adj in ADJACENT_COORDINATES]
        adjacent = [point for point in adjacent if self.point_valid(point)]
        adjacent_tiles = [self[point] for point in adjacent]
        return adjacent_tiles
    
    def possibly_connected(self, tile: MapTile) -> set[MapTile]:
        point = tile.point
        connected_coords = [point + coords for coords in CONNECTS_TO[tile.symbol]] # type: ignore
        connected_coords = [point for point in connected_coords if self.point_valid(point)]
        return set(self[point] for point in connected_coords)
        
    def get_connected(self, tile: MapTile) -> list[MapTile]:
        connected = []
        tile_connects_to = self.possibly_connected(tile)
        for other_tile in tile_connects_to:
            other_connected_to = self.possibly_connected(other_tile)
            if tile in other_connected_to:
                connected.append(other_tile)
        return connected
    
    def connected(self, tile1: MapTile, tile2: MapTile) -> bool:
        tile1_connects_to = self.possibly_connected(tile1)
        if tile2 in tile1_connects_to:
            tile2_connects_to = self.possibly_connected(tile2)
            return tile1 in tile2_connects_to
        return False

In [217]:
def ensure_is_symbol(char: str) -> Symbol:
    possible_values = get_args(Symbol)
    if char in possible_values:
        return cast(Symbol, char)
    raise ValueError(f"Wrong character: {char}. Should be one of {possible_values}")

def parse_map(data: str) -> Map:
    return Map(
        [
            [
                MapTile(ensure_is_symbol(char), Point(y, x)) for x, char in enumerate(list(row))
            ]
            for y, row in enumerate(data.splitlines())
        ]
    )

def print_map(map: Map):
    for row in map:
        print("[", *row, "]")

In [218]:
with open(f"10/{FILE}") as f:
    data = f.read()
map = parse_map(data)

In [219]:
print_map(map)

[ . . . . . ]
[ . S - 7 . ]
[ . | . | . ]
[ . L - J . ]
[ . . . . . ]


## Coordinates
Coordinates are assumed to be (y, x) with (0, 0) in the bottom left, y increasing down, and x increasing right.

In [220]:
def find_starting_point(map: Map) -> MapTile:
    for tile in map.all_tiles():
        if tile.symbol == STARTING_CHAR:
            return tile
    raise ValueError(f"Incorrect map, no starting char: {STARTING_CHAR}")

In [224]:
def starting_tile_type(map: Map):
    starting_tile = find_starting_point(map)
    adjacent_to_start = map.get_adjacent(starting_tile)
    connected_to_start: list[MapTile] = []
    for tile in adjacent_to_start:
        possibly_connected = map.possibly_connected(tile)
        if starting_tile in possibly_connected:
            connected_to_start.append(tile)
    coord_diffs = [tile.point - starting_tile.point for tile in connected_to_start]
    start_type = CONNECTED_TO_SYMBOL[tuple(sorted(coord_diffs))]
    return MapTile(start_type, starting_tile.point, starting_tile.is_loop, starting_tile.is_added)

starting_tile_replacement = starting_tile_type(map)
map[starting_tile_replacement.point] = starting_tile_replacement

ValueError: Incorrect map, no starting char: S

In [ ]:
starting_tile = starting_tile_replacement
connected_to_start = map.get_connected(starting_tile)

In [222]:
# starting_tile = find_starting_point(map)
# adjacent_to_start = map.get_adjacent(starting_tile)
# connected_to_start: list[MapTile] = []
# for tile in adjacent_to_start:
#     possibly_connected = map.possibly_connected(tile)
#     if starting_tile in possibly_connected:
#         connected_to_start.append(tile)
# connected_to_start

In [223]:
visited = {starting_tile: 0}
to_visit = [(1, connected) for connected in connected_to_start]

while len(to_visit) != 0:
    (distance, tile), *to_visit = to_visit
    if tile not in visited:
        visited[tile] = distance
        map[tile.point] = tile.set_loop(True)
        connected = map.get_connected(tile)
        for connected_tile in connected:
            if connected_tile not in visited:
                to_visit.append((distance+1, connected_tile))

Point(y=2, x=1) |


TypeError: list indices must be integers or slices, not Point

In [ ]:
def print_visited(map: Map, visited: dict[MapTile, int]):
    for row in map:
        vis = []
        for tile in row:
            if tile in visited:
                vis.append(visited[tile])
            else:
                vis.append(".")
        print("[", *vis, "]")

In [ ]:
visited

{MapTile(S @ (y=1, x=1)): 0,
 MapTile(| @ (y=2, x=1)): 1,
 MapTile(- @ (y=1, x=2)): 1,
 MapTile(L @ (y=3, x=1)): 2,
 MapTile(7 @ (y=1, x=3)): 2,
 MapTile(- @ (y=3, x=2)): 3,
 MapTile(| @ (y=2, x=3)): 3,
 MapTile(J @ (y=3, x=3)): 4}

In [ ]:
print_visited(map, visited)

[ . . . . . ]
[ . 0 1 2 . ]
[ . 1 . 3 . ]
[ . 2 3 4 . ]
[ . . . . . ]


In [ ]:
max(visited.values())

4

In [ ]:
adjacent = map.get_adjacent(starting_tile)
adjacent

[MapTile(. @ (y=0, x=1)),
 MapTile(| @ (y=2, x=1)),
 MapTile(. @ (y=1, x=0)),
 MapTile(- @ (y=1, x=2))]

In [ ]:
map.connected(map[2][1], map[3][1])

True

In [ ]:
from itertools import pairwise

def expand_map(map: Map):
    new_rows = []
    for row in map:
        for tile1, tile2 in pairwise(row):
            

SyntaxError: incomplete input (846396297.py, line 7)

In [ ]:

Point(-1,  0) - Point( 1,  0),

TypeError: unsupported operand type(s) for -: 'Point' and 'Point'